In [ ]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
%matplotlib inline 
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import chi2
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
import re
from keras.preprocessing import sequence
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

In [ ]:
# Set your own project id here
PROJECT_ID = 'your-google-cloud-project'
from google.cloud import storage
storage_client = storage.Client(project=PROJECT_ID)

In [ ]:
names=['URL','Category']
#df=pd.read_csv( "../input/website-classification-using-url/URL Classification.csv")
#df=pd.read_csv('../input/Website classification using URL/URL Classification.csv')
df=pd.read_csv('../input/website-classification-using-url/URL Classification.csv',names=names, na_filter=False)

In [ ]:
"""import re
def create_dataset(text):
  urls = list()
  for rows in text:
      row = re.sub('[^a-zA-Z]', ' ', rows)
      row = row.split()
      row = ' '.join(row[2:9])
      urls.append(row)
  print('this is URL',df['URL'][1004:1008])
  print('-'*30)
  print('this is processed urls\n', urls[1004:1008])
  dataset = pd.DataFrame(list(zip(urls, df['Category'])), columns = ['URL', 'Category'])
  return dataset

dataset = create_dataset(text = df['URL'])
dataset[:2]"""

In [ ]:
dataset = df[:]
adult = dataset[1:2000]
arts = dataset[50000:52000]
business = dataset[520000:522000]
computers = dataset[535300:537300]
games = dataset[650000:652000]
health = dataset[710000:712000]
home =  dataset[764200:766200]
kids =  dataset[793080:795080]
news =  dataset[839730:841730]
recreation =  dataset[850000:852000]
reference =  dataset[955250:957250]
science =  dataset[1013000:1015000]
shopping =  dataset[1143000:1145000]
society =  dataset[1293000:1295000]
sports =  dataset[1492000:1494000]

test_data = pd.concat([adult, arts, business, computers, games, health, home, 
              kids, news, recreation, reference,science,shopping, society, sports], axis=0)

dataset.drop(dataset.index[1:2000],inplace= True)
dataset.drop(dataset.index[50000:52000],inplace= True)
dataset.drop(dataset.index[520000:522000],inplace= True)
dataset.drop(dataset.index[535300:537300],inplace= True)
dataset.drop(dataset.index[650000:652000],inplace= True)
dataset.drop(dataset.index[710000:712000],inplace= True)
dataset.drop(dataset.index[764200:766200],inplace= True)
dataset.drop(dataset.index[793080:795080],inplace= True)
dataset.drop(dataset.index[839730:841730],inplace= True)
dataset.drop(dataset.index[850000:852000],inplace= True)
dataset.drop(dataset.index[955250:957250],inplace= True)
dataset.drop(dataset.index[1013000:1015000],inplace= True)
dataset.drop(dataset.index[1143000:1145000],inplace= True)
dataset.drop(dataset.index[1293000:1295000],inplace= True)
dataset.drop(dataset.index[1492000:1494000],inplace= True)
dataset.tail()

In [ ]:
import seaborn as sns
ax = sns.countplot(y="Category",  data=df )
plt.title("Visualization of the dataset", y=1.01, fontsize=20)
plt.ylabel("Name of the Category", labelpad=15)
plt.xlabel("Number of Categories of URLs", labelpad=15)
df.shape

In [ ]:
ax = sns.countplot(y = "Category",  data = dataset )
plt.title("Visualization of the train dataset", y=1.01, fontsize=20)
plt.ylabel("Name of the Category", labelpad=15)
plt.xlabel("Number of Categories of URLs", labelpad=15)
dataset.shape

In [ ]:
ax = sns.countplot(y = "Category",  data = test_data , color = 'gray')
plt.title("Visualization of the test dataset", y=1.01, fontsize=20)
plt.ylabel("Name of the Category", labelpad=15)
plt.xlabel("Number of Categories of URLs", labelpad=15)

In [ ]:
dataset = pd.get_dummies(dataset  ,prefix='Category', columns = ['Category'])
test_data = pd.get_dummies(test_data  ,prefix='Category', columns = ['Category'])
#print('df', df[:2])
backup_df = df
#print('df1', backup_df[:2])
#print('df2', df2[:2])
dataset[:2]
test_data[:2]

In [ ]:
X_train=dataset['URL']
y_train=dataset.iloc[: , 1:16].values
print(y_train)
print( 'y_train shape' , y_train.shape)

X_test=test_data['URL']
y_test=test_data.iloc[: , 1:16].values
print( 'y_test shape' , y_test.shape)

import re
import nltk
from sklearn.pipeline import Pipeline
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
    
stemmed_count_vect = StemmedCountVectorizer(stop_words='english', ngram_range=(2,2))

gs_clf = Pipeline([('vect', stemmed_count_vect),
                   ('tfidf', TfidfTransformer()),
                   ('clf', SGDClassifier(loss='perceptron', penalty='l2',
                    alpha =1e-4 , max_iter=20 ,tol=None)),
   ])
gs_clf = gs_clf.fit(X_train, y_train)


**This is for understandind the basics of stemming it transforms the words to its root**
* >  stem_vectorizer = stemmed_count_vect
* > text1 = 'ï am so much bored because of your meaningless behaviour'
* > print(stem_vectorizer.fit_transform([text1]))
* > print(stem_vectorizer.get_feature_names())

In [ ]:
from keras.preprocessing.text import Tokenizer
def create_and_train_tokenizer(texts):
    tokenizer=Tokenizer()
    tokenizer.fit_on_texts(texts)
    return tokenizer

from keras.preprocessing.sequence import pad_sequences
def encode_reviews(tokenizer, max_length, docs):
    encoded=tokenizer.texts_to_sequences(docs) 
    padded=pad_sequences(encoded, maxlen=max_length, padding="post")
    return padded

tokenizer=create_and_train_tokenizer(texts = X_train)
vocab_size=len(tokenizer.word_index) + 1
print("Vocabulary size:", vocab_size)

max_length=max([len(row.split()) for row in X_train])
print("Maximum length:",max_length)

X_train_encoded = encode_reviews(tokenizer, max_length, X_train)
X_test_encoded = encode_reviews(tokenizer, max_length, X_test)
print('x_train shape:', X_train_encoded.shape)
print('x_test shape:', X_test_encoded.shape)

In [ ]:
X_train_encoded[:2]

In [ ]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
from keras import layers, models
from keras.callbacks import EarlyStopping
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout

In [ ]:
from keras import layers, models
import tensorflow as tf
from tensorflow.keras import regularizers
from keras.regularizers import l2
# detect the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

# use TPUStrategy scope to define model
with strategy.scope():
    def create_embedding_model(vocab_size, max_length):
        model=models.Sequential()
        model.add(layers.Embedding(vocab_size, 100, input_length=max_length))

        model.add(layers.Conv1D(256, 2 , activation="relu"))
        #model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling1D())

        model.add(layers.Conv1D(256, 2, activation="relu"))
        #model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling1D())
        
        model.add(layers.Conv1D(256, 2, activation="relu"))
        #model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling1D())

        model.add(layers.Flatten())
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(128, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation="relu"))
        
        model.add(layers.Dense(15,  activation="softmax"))   
        return model

    
embedding_model = create_embedding_model(vocab_size=vocab_size , max_length=max_length)
embedding_model.summary()

In [ ]:
from keras.optimizers import SGD
opt = SGD(lr=0.01, momentum=0.9)
embedding_model.compile(loss='categorical_crossentropy',
              optimizer= opt,
              metrics=['accuracy'])
    
#earlyStopping = EarlyStopping(monitor="val_accuracy", patience=1)
modelHistory = embedding_model.fit(X_train_encoded, 
                                   y_train, 
                                   validation_data=(X_test_encoded, y_test),
                                   epochs= 40)              

In [ ]:
print(len(y_train) + len(y_test))
print(len(X_train_encoded))

In [ ]:
_, acc = embedding_model.evaluate(X_train_encoded, y_train, verbose=0)
print("Train accuracy:{:.2f}".format(acc*100))
_,acc= embedding_model.evaluate(X_test_encoded, y_test, verbose=0)
print("Test accuracy:{:.2f}".format(acc*100))

In [ ]:
# show a nicely formatted classification report
y_pred = (embedding_model.predict(X_test_encoded))
y_pred = (y_pred > 0.5) 

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, digits = 4))

In [ ]:
import tensorflow as tf
#saving the model
embedding_model.save('classification_model(opt(2,2)).h5')
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('classification_model(opt(2,2)).h5')
# Show the model architecture

In [ ]:
from keras.callbacks import History 
history = History()
try:
    loss_train = modelHistory.history['loss']
    loss_val = modelHistory.history['val_loss']
    epochs = range(1,41)
    plt.plot(epochs, loss_train, 'g', label='Training loss')
    plt.plot(epochs, loss_val, 'b', label='validation loss')
    plt.title('Training and Validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(),
    plt.show()
except:
    pass

In [ ]:
try:
    loss_train = modelHistory.history['accuracy']
    loss_val = modelHistory.history['val_accuracy']
    epochs = range(1,41)
    plt.plot(epochs, loss_train, 'g', label='Training accuracy')
    plt.plot(epochs, loss_val, 'b', label='validation accuracy')
    plt.title('Training and Validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
except:
    pass

In [ ]:
from nltk.tokenize import word_tokenize 
url = ['http://www.bdnews24.com', 'http://www.bdnews24.com',  'http://www.yuzutree.com', 'http://www.fcbarcelona.com']
#,'http://www.bdnews24.com', 'http://www.yuzutree.com'
# url1 = perform_stemming(websites = url)
output = list()
word1 = list()
def processed_urls(website):
    for url in website:
        if len(url)>20:
            url_data = url[:20]
        else:
            url_data = url
            
        #word1 = ' '.join(url_data)
        output.append(url_data)
        
    return output

from keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()
processed_url = processed_urls(url)
padded = encode_reviews(tokenizer, max_length ,processed_url)

categories = ['adult', 'arts', 'business', 'computers', 'games', 'health', 'home', 
              'kids', 'news', 'recreation', 'reference','science','shopping', 'society', 'sports']

y_pred1= embedding_model.predict(padded)
count  = 0
category = 0
for numbers in y_pred1:
  max_number = 1e-10
  count = 0;
  for number in numbers:
    count+=1
    if number > max_number:
      max_number = number;
      category = count
  print('category is :', categories[category-1:category])

print(category)
#y_pred
#from sklearn.metrics import classification_report
#print(classification_report(y_test, y_pred, digits = 4))